# ライブラリ

In [ ]:
import pandas as pd
!pip install skyfield
from skyfield.api import Topos, load
from datetime import timedelta, timezone
!pip install osmnx
import osmnx as ox


# 衛星の可視タイミングと位置の計算

## TLEから方位と仰角を取得

### CSVからStarlinkのTLEを取得（無駄）

In [ ]:
path = "https://celestrak.org/NORAD/elements/supplemental/sup-gp.php?FILE=starlink&FORMAT=csv"
df = pd.read_csv(path, encoding="utf-8", header=0)
df


,OBJECT_NAME,OBJECT_ID,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,RMS,DATA_SOURCE
0,STARLINK-1008,2019-074B,2025-10-04T19:29:42.000029,15.063875,0.000139,53.0530,166.3809,86.2494,36.2778,0,C,44714,277,1,6.670100e-04,0.000100,0,0.247,SpaceX-E
1,STARLINK-1010,2019-074D,2025-10-04T19:28:41.999981,15.837407,0.000514,53.0471,107.9739,101.9090,15.6907,0,C,44716,277,1,1.029200e-03,0.002357,0,0.277,SpaceX-E
2,STARLINK-1011,2019-074E,2025-10-04T19:39:41.999990,15.735676,0.000669,53.0489,162.7819,114.3264,65.5337,0,C,44717,277,1,1.249400e-03,0.001802,0,0.266,SpaceX-E
3,STARLINK-1012,2019-074F,2025-10-04T19:26:41.999971,15.063922,0.000166,53.0531,166.3914,80.5263,150.7072,0,C,44718,277,1,8.365700e-04,0.000125,0,0.250,SpaceX-E
4,STARLINK-1015,2019-074J,2025-10-04T01:53:41.999971,16.605017,0.000998,53.0309,96.2283,290.3180,31.4545,0,C,44721,277,1,-2.741500e-07,-0.001089,0,73.213,SpaceX-E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8559,STARLINK-35462,2025-223Z,2025-10-04T17:44:42.000000,16.036918,0.000917,53.1569,308.4436,261.3089,23.5336,0,C,799500417,277,1,1.497900e-03,0.010509,0,0.302,SpaceX-E
8560,STARLINK-35453,2025-223AA,2025-10-04T17:45:41.999962,16.036798,0.000918,53.1570,308.4401,261.2504,27.5866,0,C,799500418,277,1,1.473700e-03,0.010320,0,0.301,SpaceX-E
8561,STARLINK-35418,2025-223AB,2025-10-04T17:46:42.000010,16.036958,0.000919,53.1570,308.4363,260.9914,31.9082,0,C,799500419,277,1,1.461600e-03,0.010241,0,0.301,SpaceX-E
8562,STARLINK-35445,2025-223AC,2025-10-04T17:41:42.000029,16.036747,0.000923,53.1571,308.4545,260.9882,11.8238,0,C,799500420,277,1,1.477600e-03,0.010346,0,0.302,SpaceX-E


### ISSのみ

In [ ]:
ts = load.timescale()
t0 = ts.now()

osaka = Topos('34.6914 N', '135.4917 E')

satellites = load.tle('http://celestrak.com/NORAD/elements/stations.txt')
iss = satellites['ISS (ZARYA)']

alt, az, distance = (iss - osaka).at(t0).altaz()

print('高度:{0:.1f} 度'.format(alt.degrees))
print('方位角:{0:.1f} 度'.format(az.degrees))
print('距離:{0:.0f} km'.format(distance.km))


高度:-21.6 度
方位角:302.9 度
距離:5686 km


### Starlink群をインスタンス化

In [ ]:
ts = load.timescale()
t0 = ts.now()

osaka = Topos('34.6914 N', '135.4917 E')

starlink_all_TLEs = load.tle('https://celestrak.org/NORAD/elements/gp.php?GROUP=starlink&FORMAT=tle')

# Starlink衛星
starlink_instances = []

for name, satellite_instance in starlink_all_TLEs.items():
    if 'STARLINK' in str(name):
        starlink_instances.append(satellite_instance)

print(f"取得したStarlink衛星の数: {len(starlink_instances)}機")

for instance in starlink_instances[-1:]:
    alt, az, distance = (instance - osaka).at(t0).altaz()
    print(f"{instance.name}: 高度: {alt.degrees:.1f} 度, 方位角: {az.degrees:.1f} 度, 距離: {distance.km} km")


取得したStarlink衛星の数: 8385機
STARLINK-34749: 高度: -28.8 度, 方位角: 317.6 度, 距離: 6907.299666630773 km


## 天球内のパスを発見するメソッド
find_eventsメソッドを使えば，指定期間内にインスタンスが任意地点上空を通るパスのうち，指定仰角以上となる全てのパスの（出現時刻・最大仰角となる時刻・没時刻）を一度に得ることができる．

In [ ]:
ts = load.timescale()
t0 = ts.now()
t1 = ts.utc(t0.utc_datetime() + timedelta(days=1))
tz = timezone(timedelta(hours=9))

t, events = starlink_instances[0].find_events(osaka, t0, t1, altitude_degrees=10.0)

for ti, event in zip(t, events):
    if event == 0:
        print('見え始め', ti.astimezone(tz).strftime('%Y-%m-%d %H:%M:%S'), 'JST')
    if event == 1:
        print('最大仰角', ti.astimezone(tz).strftime('%Y-%m-%d %H:%M:%S'), 'JST')
    if event == 2:
        print('見え終わり', ti.astimezone(tz).strftime('%Y-%m-%d %H:%M:%S'), 'JST')
        print()


見え始め 2025-10-05 18:46:41 JST
最大仰角 2025-10-05 18:50:16 JST
見え終わり 2025-10-05 18:53:49 JST

見え始め 2025-10-05 20:26:01 JST
最大仰角 2025-10-05 20:29:48 JST
見え終わり 2025-10-05 20:33:33 JST

見え始め 2025-10-06 10:19:33 JST
最大仰角 2025-10-06 10:21:32 JST
見え終わり 2025-10-06 10:23:31 JST

見え始め 2025-10-06 11:56:31 JST
最大仰角 2025-10-06 12:00:35 JST
見え終わり 2025-10-06 12:04:41 JST

見え始め 2025-10-06 13:38:56 JST
最大仰角 2025-10-06 13:40:47 JST
見え終わり 2025-10-06 13:42:39 JST



## 太陽と可視タイミングの関係

In [ ]:
eph = load('de421.bsp')
sun, earth = eph['sun'], eph['earth']

ts = load.timescale()
t0 = ts.now()
t1 = ts.utc(t0.utc_datetime() + timedelta(days=1))
tz = timezone(timedelta(hours=9))

t, events = starlink_instances[0].find_events(osaka, t0, t1, altitude_degrees=10.0)

sun_alt = (earth + osaka).at(t).observe(sun).apparent().altaz()[0].degrees
sun_lit = starlink_instances[-1].at(t).is_sunlit(eph)

for ti, event, s_alt, s_lit in zip(t, events, sun_alt, sun_lit):
    if s_alt < -6 and s_lit == True:
        if event == 0:
            print('見え始め', ti.astimezone(tz).strftime('%Y-%m-%d %H:%M:%S'), 'JST')
        if event == 1:
            print('最大仰角', ti.astimezone(tz).strftime('%Y-%m-%d %H:%M:%S'), 'JST')
        if event == 2:
            print('見え終わり', ti.astimezone(tz).strftime('%Y-%m-%d %H:%M:%S'), 'JST')
            print()


見え始め 2025-10-05 18:46:41 JST
最大仰角 2025-10-05 18:50:16 JST


# 観測候補地点のリストアップ

In [ ]:
# Pandasの表示設定（全ての列を表示）
pd.set_option('display.max_columns', None)

# 探索の中心点と範囲を設定
center_point = (34.4223, 132.7441)
search_radius = 15000

# 抽出したい場所のOSMタグを定義
# OSMタグはキーとバリューの組み合わせで場所の種類を定義している．
tags = {
    'leisure': 'park',     # 公園
    'amenity': 'parking',  # 駐車場
    'tourism': 'viewpoint' # 展望台
}

# OSMからデータを取得（戻り値はGeoDataFrame）
gdf = ox.features_from_point(center_point, tags, dist=search_radius)
print(f"{len(gdf)}件の候補が見つかりました．")

# name列が存在しない，または名前が空欄のものを除外
# 名前がないものは，大学の駐車場やコンビニだったりする．
candidate_list = gdf[gdf['name'].notna()].copy()

# 緯度経度情報をgeometry列から抽出
candidate_list['latitude'] = candidate_list['geometry'].apply(lambda p: p.centroid.y)
candidate_list['longitude'] = candidate_list['geometry'].apply(lambda p: p.centroid.x)

# 表示する列を絞り込み
candidate_list = candidate_list[['name', 'latitude', 'longitude', 'leisure', 'amenity', 'tourism']]

print(f"うち，名称が登録されているのは{len(candidate_list)}件です．")

candidate_list


774件の候補が見つかりました．
うち，名称が登録されているのは87件です．


name   latitude   longitude leisure amenity  \
element id                                                               
node    1910014046        吾妻子の滝  34.396653  132.739186     NaN     NaN   
        3623306448        生態実験園  34.403021  132.716421    park     NaN   
        4005445021       スペイン広場  34.398516  132.711539    park     NaN   
        4285799112  移設古墳群と文学の小庭  34.403793  132.712838    park     NaN   
        4926542547        稲荷坂公園  34.456948  132.822376    park     NaN   
...                         ...        ...         ...     ...     ...   
way     1272167905      中野東第三公園  34.411214  132.585066    park     NaN   
        1359791714         中央公園  34.342807  132.902545    park     NaN   
        1366170827      寺家塚の峠公園  34.438556  132.719253    park     NaN   
        1422969838     寺家駅前３号公園  34.440171  132.722405    park     NaN   
        1423245173     寺家貞松第二公園  34.433484  132.727359    park     NaN   

                      tourism  
element id                     
node    1910014046  viewpoint  
        3623306448        NaN  
        4005445021        NaN  
        4285799112        NaN  
        4926542547        NaN  
...                       ...  
way     1272167905        NaN  
        1359791714        NaN  
        1366170827        NaN  
        1422969838        NaN  
        1423245173        NaN  

[87 rows x 6 columns]